# Chapter8 어텐션

## 8.1 어텐션 구조

### 8.1.1 seq2seq의 문제점

### 8.1.2 Encoder 개선

### 8.1.3 Decoder 개선1

### 8.1.4 Decoder 개선2

### 8.1.5 Decoder 개선3

## 8.2 어텐션을 갖춘 seq2seq 구현

### 8.2.1 Encoder 구현

### 8.2.2 Decoder 구현

### 8.2.3 seq2seq 구현

## 8.3 어텐션 평가

### 8.3.1 날짜 형식 변환 문제

### 8.3.2 어텐션을 갖춘 seq2seq의 학습

### 8.3.3 어텐션 시각화

## 8.4 어텐션에 관한 남은 이야기

### 8.4.1 양방향 RNN

### 8.4.2 Attention 계층 사용 방법

### 8.4.3 seq2seq 심층화와 skip 연결

## 8.5 어텐션 응용

### 8.5.1 구글 신경망 기계 번역 (GNMT)

### 8.5.2 트랜스포머

### 8.5.3 뉴럴 튜링 머신 (NTM)